In [1]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd

In [2]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'ws4_data_1649611737.751587'

In [3]:
TABLE_ID

'ws4_data_1649611737.751587'

### Utils functions

In [4]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

### Connecting to BigTable

In [5]:
client = bigtable.Client.from_service_account_json('../../unbosque-service-account.json', admin = True)

In [6]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


### Creating a table

In [7]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the ws4_data_1649611737.751587 table
ERROR: Table ws4_data_1649611737.751587 already exists


In [8]:
print('Creating all family columns')

plant_cfId = 'plant'
plant_cf = table.column_family(plant_cfId)
plant_cf.create()

generation_cfId = 'generation'
generation_cf = table.column_family(generation_cfId)
generation_cf.create()

weather_cfId = 'weather'
weather_cf = table.column_family(weather_cfId)
weather_cf.create()

print('Columns created! Please wait...\n')

Creating all family columns


AlreadyExists: 409 Cannot create existing family plant

In [9]:
#REQ1: Cree el conjunto de instrucciones necesarias para cargar a Python e insertar en BigTable todos los registros contenidos
#en los archivos de clima y generación eléctrica para las dos plantas de las que se dispone información
print('\nLoading data from CSV datasets...')
Plant1_GenData = pd.read_csv('./Plant_1_Generation_Data.csv')
Plant2_GenData = pd.read_csv('./Plant_2_Generation_Data.csv')
Plant1_WeatherData = pd.read_csv('./Plant_1_Weather_Sensor_Data.csv')
Plant2_WeatherData = pd.read_csv('./Plant_2_Weather_Sensor_Data.csv')
Plant1_jsonG = Plant1_GenData.to_dict(orient='records')
Plant1_jsonW = Plant1_WeatherData.to_dict(orient='records')
Plant2_jsonG = Plant2_GenData.to_dict(orient='records')
Plant2_jsonW = Plant2_WeatherData.to_dict(orient='records')
print('Data ready! Please wait...')
    


Loading data from CSV datasets...
Data ready! Please wait...


In [10]:
dt = dt.utcnow()
rows = []

print('Writing to the table')
for weather in Plant2_jsonW:
    
    row_key = 'date_time#{}'.format(weather['DATE_TIME']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(plant_cfId, 'ambient_temperature'.encode(), str(weather['AMBIENT_TEMPERATURE']), timestamp = dt)
    row.set_cell(plant_cfId, 'module_temperature'.encode(), str(weather['MODULE_TEMPERATURE']), timestamp = dt)
    row.set_cell(plant_cfId, 'irradiation'.encode(), str(weather['IRRADIATION']), timestamp = dt)
    
    
    
    rows.append(row)

table.mutate_rows(rows)

Writing to the table


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

### Reading data

In [18]:
def read_row_range():

    row_set = RowSet()
    row_set.add_row_range_from_keys(
        start_key=b"date_time#2020-06-01 06:00:00", end_key=b"date_time#2020-06-01 12:00:00"
    )

    filter2=row_filters.ColumnQualifierRegexFilter(b"ambient_temperature")
   
    rows = table.read_rows(row_set=row_set,filter_=filter2)
    for row in rows:
        print_row(row)

read_row_range()

Reading data for date_time#2020-06-01 06:00:00:
Column Family plant
	ambient_temperature: 23.470155033333327 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 06:15:00:
Column Family plant
	ambient_temperature: 23.67332510344828 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 06:30:00:
Column Family plant
	ambient_temperature: 23.979217633333327 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 06:45:00:
Column Family plant
	ambient_temperature: 24.209690400000003 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 07:00:00:
Column Family plant
	ambient_temperature: 24.567252344827587 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 07:15:00:
Column Family plant
	ambient_temperature: 24.835614033333336 @2022-04-11 04:19:06.682000+00:00

Reading data for date_time#2020-06-01 07:30:00:
Column Family plant
	ambient_temperature: 25.36163324137931 @2022-04-11 04:19:06.682000+0

In [14]:
#REQ 2: ¿Cuáles fueron las diferentes medidas de temperatura ambiente para la planta 2 con fecha del 1 de junio desde las 6 de
#la mañana al mediodía? La consulta debe traer solamente las filas y columnas necesarias.
Plant2_WeatherData.dtypes

DATE_TIME              object
PLANT_ID                int64
SOURCE_KEY             object
AMBIENT_TEMPERATURE    object
MODULE_TEMPERATURE     object
IRRADIATION            object
dtype: object